In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
import requests
import pandas as pd

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')
service = Service("/Users/tumendemberelshalkhaan/Downloads/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://appgateway.drc.ohio.gov/OffenderSearch')
ids = []

In [ ]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')
select = soup.find('select', {'name': 'CntyCommitment'})
options = select.find_all('option')
option_text = [option.text for option in options]
counties = option_text[1:]

In [ ]:
for county in counties:
    
    driver.find_element("xpath", ".//*[@id='rdoI']/input").click()
    select = Select(driver.find_element("name", 'CntyCommitment'))
    select.select_by_visible_text(county)
    driver.find_element("xpath", ".//*[@value='Search']").click()

    current_button = driver.find_element("xpath", "//input[@type='button' and @disabled]")

    links = driver.find_elements("xpath", "//a[starts-with(@href, '/OffenderSearch/Search/Details/') and contains(@href, '/')]")
    for link in links:
        if link.text != "":
            ids.append(link.text)

    # Loop through the buttons until there is no next button
    while True:
        # Find the next enabled button relative to the current button
        try:
            next_button = current_button.find_element("xpath", "./following-sibling::input[@type='button' and not(@disabled)]")
        except:
            break
        # Click the next button
        next_button.click()
        links = driver.find_elements("xpath", "//a[starts-with(@href, '/OffenderSearch/Search/Details/') and contains(@href, '/')]")
        for link in links:
            if link.text != "":
                ids.append(link.text)
        # Set the current button to the next button

        current_button = driver.find_element("xpath", "//input[@type='button' and @disabled]")

    driver.get('https://appgateway.drc.ohio.gov/OffenderSearch')

In [ ]:
oh = pd.DataFrame(columns=[])

In [ ]:
failed_ids = []

for id in ids:
    try:
        url = ('https://appgateway.drc.ohio.gov/OffenderSearch/Search/Details/' + id)

        response = requests.get(url)

        soup = BeautifulSoup(response.text, 'html.parser')

        name_element = soup.find('div', {'class': 'panel-heading text-center'})
        name = name_element.text.strip()

        dd = soup.find_all('dd')
        dob = dd[1].text.strip()
        gender = dd[2].text.strip()
        race = dd[3].text.strip()
        fac = dd[5].text.strip()

        charges = []
        charge_elements = soup.find_all('div', {'class': "h4"})
        for charge in charge_elements:
            charges.append(charge.text.strip())

        codes = []
        code_elements = soup.find_all('label', {'for': 'ORC:'})
        for code in code_elements:
            codes.append(code.next_sibling.strip())

        sentences = []
        sentence_elements = soup.find_all('label', {'for': ['Definite_Term_Yrs:', 'Max_Yrs:', 'SB201_Max_Yrs:', 'SB201_Mand_Max_Yrs:', 'Mand_AI_Yrs:', 'SB201_Min_Yrs:', 'SB201_Mand_Min_Yrs:']})
        for sentence in sentence_elements:
            sentences.append(sentence.next_sibling.strip())

        counties = []
        county_elements = soup.find_all('label', {'for': 'Committing_County:'})
        for county in county_elements:
            counties.append(county.next_sibling.strip())

        sendates = []
        sendate_elements = soup.find_all('label', {'for': 'Effective_Sentence_Date:'})
        for sendate in sendate_elements:
            sendates.append(sendate.next_sibling.strip())

        counts = []
        count_elements = soup.find_all('label', {'for': 'Counts:'})
        for count in count_elements:
            counts.append(count.next_sibling.strip())

        crimes = [list(pair) for pair in zip(charges, codes, sentences, counties, sendates, counts)]

        rows = []
        for pair in crimes:

            row = {
                'ID': id,
                'Name': name,
                'Date Of Birth': dob,
                'Gender': gender,
                'Race': race,
                'County': pair[3],
                'Facility': fac,
                'Offense Description': pair[0],
                'Offense Code': pair[1],
                'Sentence Date': pair[4],
                'Sentence Length': pair[2],
                'Counts': pair[5]
            }
            rows.append(row)
            
        oh = pd.concat([oh, pd.DataFrame(rows)])
        
    except Exception as e:
        print(f"Failed to scrape {id}. Error: {e}")
        failed_ids.append(id)
        continue

In [ ]:
oh.to_csv('/Users/tumendemberelshalkhaan/Desktop/felony_murder/raw_data/Ohio_Raw.csv', index=False)